# NTSB Modeling XG BOOST

## Notes

1. Unknowns all filled with nan values
2. Rebalancing done as random under sampling and over sampling
3. 3 categories, normalized

## Create data strings

In [1]:
import pandas as pd
import numpy as np
import csv
import re
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
data = pd.read_csv('labeled_clean_ntsb.csv')

C:\Users\htwal\AppData\Local\Temp\ipykernel_14084\421915254.py:1: DtypeWarning: Columns (3,17,18,23,44,45,78,80,81,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('labeled_clean_ntsb.csv')


In [3]:
time1 = pd.read_csv('flight_time_pt.csv')
time2 = pd.read_csv('flight_time_pre.csv')

In [4]:
time = pd.concat([time1, time2])
time = time.drop(['Aircraft_Key', 'crew_no'], axis=1)

In [5]:
time['ev_id'] = time['ev_id'].str.lower()

In [6]:
# create a pivot table:
time_pt = pd.pivot_table(time, values='flight_hours', index=['ev_id'], columns=['flight_craft'])

In [7]:
time_pt.shape

(78293, 12)

In [8]:
data = data.fillna(np.nan)
data = data.replace('', np.nan)
data = data.replace('nan', np.nan)
data = data.replace('unk', np.nan)
data = data.replace('unknown', np.nan)

In [9]:
# merge with main data set
data_set = data.merge(time_pt, left_on='aircraft_ev_id', right_on='ev_id')

In [10]:
# Create training dataset:
cols = ['Unnamed: 0', 'aircraft_ev_id', 'far_part', 'damage', 'phase_flt_spec', 'ntsb_no', 
        'ev_year', 'ev_type', 'apt_dist', 'apt_dir', 'ev_highest_injury', 'inj_tot_t', 'secondary_eng_mfgr',
        'crew_inj_level', 'inj_target', 'damage_target', 'hp_or_lbs', 'carb_fuel_injection', 'wx_src_iic', 'wx_obs_fac_id', 'wx_obs_elev', 'wx_obs_dir', 'wx_obs_dist',
        'gust_ind', 'wind_vel_ind', 'crew_category', 'pilot_flying', 'med_certf', 'INC', 'LTA', 'GLDR',
        'REL']
data_set = data_set.drop(cols, axis=1)

In [11]:
data_set = data_set.applymap(lambda x: re.sub(r'\bn\b', 'no', x) if type(x) == str else x)
data_set = data_set.applymap(lambda x: re.sub(r'\by\b', 'yes', x) if type(x) == str else x)

In [12]:
data_set = data_set.applymap(lambda x: x.strip() if type(x) == str else x)

In [13]:
data_set[data_set.select_dtypes(['object']).columns] = data_set.select_dtypes(['object']).astype('category')

In [14]:
data_set[data_set.select_dtypes(['category']).columns] = data_set.select_dtypes(['category']).apply(lambda x: x.cat.codes)

In [15]:
cat_cols = data_set.select_dtypes(include='category').columns.tolist()

### Create a rebalanced dataset

In [16]:
target = data_set['target'].astype(int)

In [17]:
target.value_counts()

3    33403
4    11968
2      589
1       92
Name: target, dtype: int64

In [18]:
data_vals = data_set.drop(['target'], axis=1)

In [19]:
# undersample 3
rus = RandomUnderSampler(sampling_strategy={3:11968}, random_state=25)
X_res, y_res = rus.fit_resample(data_vals, target)
print('Resampled dataset shape %s' % Counter(y_res))

# oversample 1, 2
ros = RandomOverSampler(random_state=33)
X_bal, y_bal = ros.fit_resample(X_res, y_res)
print('Resampled dataset shape %s' % Counter(y_bal))

Resampled dataset shape Counter({3: 11968, 4: 11968, 2: 589, 1: 92})
Resampled dataset shape Counter({1: 11968, 2: 11968, 3: 11968, 4: 11968})


In [20]:
print(len(X_bal))
print(len(y_bal))

47872
47872


## Normalization

In [21]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_bal)

## Model Setup

In [22]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_bal, test_size=0.20, random_state=33)

In [23]:
xg_boost = HistGradientBoostingClassifier(categorical_features=cat_cols).fit(X_train, y_train)

In [24]:
xg_boost.score(X_train, y_train)

0.92218711648432

In [25]:
y_pred = xg_boost.predict(X_test)

In [26]:
f1 = metrics.f1_score(y_test, y_pred, average='weighted')
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred, average='weighted')
recall = metrics.recall_score(y_test, y_pred, average='weighted')

In [29]:
print(f'f1 score: {(f1*100).round(2)}')
print(f'Accuracy: {(accuracy*100).round(2)}')
print(f'Precision: {(precision*100).round(2)}')
print(f'Recall: {(recall*100).round(2)}')

f1 score: 84.4
Accuracy: 84.58
Precision: 84.37
Recall: 84.58


In [28]:
cm = metrics.confusion_matrix(y_test, y_pred)
metrics.ConfusionMatrixDisplay(cm).plot()